In [3]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML
import csv

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.colors import DEFAULT_PLOTLY_COLORS

from simulator import simulate as _simulate
from simulator_plotting import *

init_notebook_mode(connected=True)

def simulate(landscape, **kwargs):
    # wrapper function for the c extension's simulate

    # landscape could contain Series or ndarrays. convert them to lists.
    try:
        landscape = [i if isinstance(i, list) else i.tolist() for i in landscape]
    except (TypeError, AttributeError) as err:
        raise TypeError('invalid landscape') from err
    
    return _simulate(landscape, **kwargs)


pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [41]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    times = [] # T_f's
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
        times.append(results['T_f'])
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if not np.isnan(max_weight):
        dominant_path = df.iloc[df['Weight'].idxmax()]['Pathway']
    else:
        dominant_path = ""
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            'success_rate': num_successful_paths / num_paths,
            'dominant_path': dominant_path,
            'times': times
        }

In [42]:
def box_switching_ctx(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, analyses=None, mean=True, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    shapes = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    for i, f in enumerate(frequencies):
        param['frequency'] = f
        if analyses:
            analysis=analyses[f]
        else:
            analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = dict(
            type = 'box',
            #x = [f for i in range(len(times))],
            y = times,
            name = '&#120591;={}'.format(f),
            boxpoints = False,
            line = dict(width=4)
        )
        data.append(trace)
        wx.append('&#120591;={}'.format(f))
        wy.append(analysis['success_count'] / num)
        if mean and times:
            shapes.append(dict(
                type = 'line',
                xref = 'x',
                yref = 'y',
                x0 = i - 0.4,
                x1 = i + 0.4,
                y0 = np.mean(times),
                y1 = np.mean(times),
                line = {'color': DEFAULT_PLOTLY_COLORS[i % 10], 'width': 2}
                #layer = 'below'
            ))
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers',
        cliponaxis = False
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different periods, K=10<sup>{}</sup>'.format(num, name, k),
        xaxis = dict(
            title = 'Switching period',
            tickvals = ['&#120591;={}'.format(f) for f in frequencies],
            #tickfont=dict(size=10),
            ticks='outside',
            showline=True,
            mirror=True
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,1500],
            tickvals=[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500],
            ticktext=[0,'','','','',500,'','','','',1000,'','','','',1500],
            showline=True,
            mirror=True
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            scaleanchor='y',
            scaleratio=1500,
            side = 'right',
            range=[0,1.0],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False,
        ),
        showlegend = False,
        shapes = shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [52]:
p = pairs[0]
frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1]
ctx_analysis = {f : pathway_analysis(p, num=100, carrying_cap=10**9, frequency=f) for f in frequencies}

In [53]:
fig = box_switching_ctx(p, analyses=ctx_analysis)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{}.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [54]:
fig = box_switching_ctx(p, analyses=ctx_analysis, mean=False)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{} no mean.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [37]:
def box_switching_zox(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, analyses=None, mean=True, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    shapes = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    for i, f in enumerate(frequencies):
        param['frequency'] = f
        if analyses:
            analysis=analyses[f]
        else:
            analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = dict(
            type = 'box',
            #x = [f for i in range(len(times))],
            y = times,
            name = '&#120591;={}'.format(f),
            boxpoints = False,
            line = dict(width=4)
        )
        data.append(trace)
        wx.append('&#120591;={}'.format(f))
        wy.append(analysis['success_count'] / num)
        if mean and times:
            shapes.append(dict(
                type = 'line',
                xref = 'x',
                yref = 'y',
                x0 = i - 0.4,
                x1 = i + 0.4,
                y0 = np.mean(times),
                y1 = np.mean(times),
                line = {'color': DEFAULT_PLOTLY_COLORS[i % 10], 'width': 2}
                #layer = 'below'
            ))
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers',
        cliponaxis = False
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different periods, K=10<sup>{}</sup>'.format(num, name, k),
        xaxis = dict(
            title = 'Switching period',
            tickvals = ['&#120591;={}'.format(f) for f in frequencies],
            #tickfont=dict(size=10),
            ticks='outside',
            showline=True,
            mirror=True
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,250],
            tickvals=[0,25,50,75,100,125,150,175,200,225,250],
            ticktext=[0,'',50,'',100,'',150,'',200,'',250],
            showline=True,
            mirror=True
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            scaleanchor='y',
            scaleratio=250,
            side = 'right',
            range=[0,1.0],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False
        ),
        showlegend = False,
        shapes = shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [33]:
p = pairs[2]
frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1]
zox_analysis = {f : pathway_analysis(p, num=100, carrying_cap=10**9, frequency=f) for f in frequencies}

In [39]:
fig = box_switching_zox(p, analyses=zox_analysis)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{}.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [40]:
fig = box_switching_zox(p, analyses=zox_analysis, mean=False)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{} no mean.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [46]:
def box_switching_am(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, analyses=None, mean=True, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    shapes = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    for i, f in enumerate(frequencies):
        param['frequency'] = f
        if analyses:
            analysis=analyses[f]
        else:
            analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = dict(
            type = 'box',
            #x = [f for i in range(len(times))],
            y = times,
            name = '&#120591;={}'.format(f),
            boxpoints = False,
            line = dict(width=4)
        )
        data.append(trace)
        wx.append('&#120591;={}'.format(f))
        wy.append(analysis['success_count'] / num)
        if mean and times:
            shapes.append(dict(
                type = 'line',
                xref = 'x',
                yref = 'y',
                x0 = i - 0.4,
                x1 = i + 0.4,
                y0 = np.mean(times),
                y1 = np.mean(times),
                line = {'color': DEFAULT_PLOTLY_COLORS[i % 10], 'width': 2}
                #layer = 'below'
            ))
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers',
        cliponaxis = False
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different periods, K=10<sup>{}</sup>'.format(num, name, k),
        xaxis = dict(
            title = 'Switching period',
            tickvals = ['&#120591;={}'.format(f) for f in frequencies],
            #tickfont=dict(size=10),
            ticks='outside',
            showline=True,
            mirror=True
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,1500],
            tickvals=[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500],
            ticktext=[0,'','','','',500,'','','','',1000,'','','','',1500],
            showline=True,
            mirror=True
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            scaleanchor='y',
            scaleratio=1500,
            side = 'right',
            range=[0,1.0],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False,
        ),
        showlegend = False,
        shapes = shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [47]:
p = pairs[1]
frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1]
am_analysis = {f : pathway_analysis(p, num=100, carrying_cap=10**9, frequency=f) for f in frequencies}

In [48]:
fig = box_switching_am(p, analyses=am_analysis)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{}.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [49]:
fig = box_switching_am(p, analyses=am_analysis, mean=False)
iplot(fig, show_link=False)
pio.write_image(fig, '2020-03/violins/{} no mean.pdf'.format('+'.join([ls.name for ls in p])), scale=2)

In [55]:
import json

In [78]:
data = {}
num=100
for s, a in (('ctx-sam', ctx_analysis), ('am-amc', am_analysis), ('zox-cxm', zox_analysis)):
    data[s] = [{'tau': tau, 'times': val['times'], 'success_rate': val['success_count']/num} for tau, val in a.items()]

In [59]:
am_analysis

{200: {'paths':                          Pathway Successful? Greedy?  Number of appearances  \
  0                      0000,0010          No     Yes                      0   
  1  0000,0010,1010,1011,1111,1101         Yes      No                     92   
  2  0000,0010,0011,1011,1111,1101         Yes      No                      8   
  
     Weight  
  0     NaN  
  1    0.92  
  2    0.08  ,
  'global_optimum': '1101',
  'max_weight': 0.92,
  'success_count': 100,
  'success_rate': 1.0,
  'dominant_path': '0000,0010,1010,1011,1111,1101',
  'times': [774,
   742,
   761,
   776,
   774,
   771,
   748,
   736,
   777,
   759,
   766,
   774,
   747,
   771,
   735,
   775,
   761,
   770,
   766,
   615,
   757,
   748,
   751,
   746,
   780,
   759,
   772,
   729,
   742,
   760,
   721,
   753,
   764,
   774,
   775,
   745,
   757,
   743,
   767,
   760,
   728,
   743,
   731,
   774,
   763,
   772,
   762,
   749,
   760,
   758,
   769,
   772,
   748,
   754,
   763,
   7

In [79]:
with open('2020-03/violins/plot-data-2.json', 'w') as fp:
    json.dump(data, fp)

In [80]:
with open('2020-03/violins/plot-data-2.json', 'r') as fp:
    data_in = json.load(fp)

In [81]:
data_in == data

True